<a href="https://colab.research.google.com/github/mechhector/visao_computacional20212/blob/main/trabalho3/trabalho3_fe_clf_hector_leonardo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instituto Federal de Educação, Ciência e Tecnologia do Ceará

## Departamento de Indústria - *Campus* Fortaleza

## Bacharelado em Egenharia de Mecatrônica

## Visão Computacional - Prof. Pedro Pedrosa

## Trabalho 3 - Extração de Atributos e Classificação

### **Hector Leonardo Mota Moreira**


## Functions

In [ ]:
def verifyNeigh(image, point, cn, chain_list):

  '''
  Verifies the neighborhood for 4- and 8-connectivity

  [- 0 -]
  [3 • 1]  4-connected reference
  [- 2 -]

  [7 0 1]
  [6 • 2]  8-connected reference
  [5 4 3]

  '''

  # 4-connected
  if cn == 4:

    #print(point)

    # 0
    if image[point[0]-1,point[1]] == 255:
      image[point[0]-1,point[1]] = 0
      #print('0')
      chain_list.append(0)
      return (point[0]-1,point[1])

    # 1
    elif image[point[0] , point[1]+1] == 255:
      image[point[0] , point[1] + 1] = 0
      #print('1')
      chain_list.append(1)
      return ([point[0] , point[1] + 1])

    # 2
    elif image[point[0]+1 , point[1]] == 255:
      image[point[0]+1 , point[1]] = 0
      #print('2')
      chain_list.append(2)
      return ([point[0]+1 , point[1]])

    # 3
    elif image[point[0] , point [1]-1] == 255:
      image[point[0] , point[1]-1] = 0
      #print('3')
      chain_list.append(3)
      return ([point[0] , point[1]-1])

    else:
      #print('none')
      point = start_point

  
  # 8-connected     
  if cn == 8: 

    print(point)

     # 0
    if image[point[0]-1,point[1]] == 255:
      image[point[0]-1,point[1]] = 0
      print('0')
      chain_list.append(0)
      return (point[0]-1,point[1])
      
    # 1
    elif image[point[0]-1,point[1]+1] == 255:
      image[point[0]-1,point[1]+1] = 0
      print('1')
      chain_list.append(1)
      return (point[0]-1,point[1]+1)

    # 2
    elif image[point[0] , point[1]+1] == 255:
      image[point[0] , point[1]+1] = 0
      print('2')
      chain_list.append(2)
      return ([point[0] , point[1]+1])
      
    # 3
    elif image[point[0]+1,point[1]+1] == 255:
      image[point[0]+1,point[1]+1] = 0
      print('3')
      chain_list.append(3)
      return (point[0]+1,point[1]+1)

    # 4
    elif image[point[0]+1 , point[1]] == 255:
      image[point[0]+1 , point[1]] = 0
      print('4')
      chain_list.append(4)
      return ([point[0]+1 , point[1]])

    # 5
    elif image[point[0]+1,point[1]-1] == 255:
      image[point[0]+1,point[1]-1] = 0
      print('5')
      chain_list.append(5)
      return (point[0]+1,point[1]-1)

    # 6
    elif image[point[0],point[1]-1] == 255:
      image[point[0],point[1]-1] = 0
      print('6')
      chain_list.append(6)
      return (point[0],point[1]-1)

    # 7
    elif image[point[0]-1 , point [1]-1] == 255:
      image[point[0]-1 , point[1]-1] = 0
      print('7')
      chain_list.append(7)
      return ([point[0]-1 , point[1]-1])

    else:
      print('none')
      point = start_point
      
      
  else:
      return point


In [ ]:
def chainCodeNormalization(chain_code_list):

  '''

  Normalizes the chain code into the shortest lenght 

  '''

  normalized_list = []
  
  normalized_list.append(chain_code_list[0])

  proportion_ratio = len(chain_code_list) / 21

  sum = 0

  holder = proportion_ratio

  #print('proportion ratio: ',proportion_ratio)

  while len(normalized_list) < 21:

    sum = sum + int(holder)

    #print(sum)

    normalized_list.append(chain_code_list[sum])

    pr_decimal = holder % 1

    holder = proportion_ratio + pr_decimal



  return normalized_list 


In [ ]:
def normalizeImage(v):
  '''
  Normalizes an image between 0 and 255
  '''


  v = (v - v.min()) / (v.max() - v.min())
  result = (v * 255).astype(np.uint8)

  #cv2_imshow(result)

  return result

In [ ]:
# HU Moments

def HU_FE(image):
        moments = cv2.moments(image.astype(np.float64))
        return np.asarray(cv2.HuMoments(moments).flatten())

# Gray Level Cooccurrency Matrix

def GLCM_FE(image):
        glcm = greycomatrix(image, [1], [0], 256, symmetric=True, normed=True)
        xs = []
        xs.append(greycoprops(glcm, 'dissimilarity')[0, 0])
        xs.append(greycoprops(glcm, 'correlation')[0, 0])
        xs.append(greycoprops(glcm, 'homogeneity')[0, 0])
        xs.append(greycoprops(glcm, 'ASM')[0, 0])
        xs.append(greycoprops(glcm, 'energy')[0, 0])
        xs.append(greycoprops(glcm, 'correlation')[0, 0])
        return np.asarray(xs);


## Imports and data loading


In [ ]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

from scipy import stats
from scipy import ndimage

from skimage.feature import greycomatrix, greycoprops, local_binary_pattern
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, cross_val_score, StratifiedShuffleSplit
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.feature import corner_fast, corner_peaks
from sklearn.cluster import spectral_clustering
from sklearn.feature_extraction import image
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier

from google.colab.patches import cv2_imshow

# Loading the dataset 

data_path = '/content/drive/MyDrive/DIP_2021.2/Trabalho 3 - DIP 2021.2/ocr_car_numbers_rotulado_1.txt'

data = np.loadtxt(data_path)

rows , cols = data.shape

print(data.shape)

(3352, 1226)


## FE Data

In [ ]:
# HU Moments Matrix

hu_moments_matrix = np.zeros((rows, 7))

for i in range(rows):
  current_row = data[i]

  current_row_image = current_row[0:cols-1].reshape(35,35) 

  current_row_normalized_image = normalizeImage(current_row_image) 

  hu_moments_current_row = HU_FE(current_row_normalized_image)

  #cv2_imshow(current_row_normalized_image)

  hu_moments_matrix[i] = hu_moments_current_row

  #print(current_row[cols-1])

In [ ]:
# GLCM Matrix

glcm_matrix = np.zeros((rows, 6))

for i in range(rows):
  current_row = data[i]

  current_row_image = current_row[0:cols-1].reshape(35,35) 

  current_row_normalized_image = normalizeImage(current_row_image) 

  glcm_current_row = GLCM_FE(current_row_normalized_image)

  #cv2_imshow(current_row_normalized_image)

  glcm_matrix[i] = glcm_current_row

  #print(current_row[cols-1])

In [ ]:
# Chain List Matrix

shortest_list = 21
shortest_list_index = 0

normalized_data_chain = np.zeros((rows, shortest_list))

print(normalized_data_chain.shape)



for i in range(rows):

  # ============================= form an image ================================
 
  current_row = data[i]

  current_row_image = current_row[0:cols-1].reshape(35,35) 

  current_row_normalized_image = normalizeImage(current_row_image) 

  #print(current_row_normalized_image.shape)

  filled_image = (ndimage.binary_fill_holes(current_row_normalized_image).astype(int))

  normalized_filled_image = normalizeImage(filled_image)

  
  # ====================== dilation to get borders =============================

  kernel = np.ones((3,3))
  dilation = cv2.dilate(normalized_filled_image,kernel,iterations = 1)

  border = dilation-normalized_filled_image 

  #cv2_imshow(border)


  # ====================== identify the start point ============================

  max_xy = np.where(border == 255)

  #print(max_xy[0][0] , max_xy[1][0])

  start_point = max_xy[0][0] , max_xy[1][0]


  # ============================== chain list ==================================

  chain_list = []

  point = verifyNeigh(border, start_point, 4, chain_list)

  while(point != start_point):

    point = verifyNeigh(border, point, 4, chain_list)

    #print(len(chain_list))

  # ===================== to find the shortest list =============================

  #if len(chain_list) < shortest_list:
  # shortest_list = len(chain_list)
  # shortest_list_index = i 

  # =========================== list normalization =============================    

  if len(chain_list) > shortest_list:
    normalized_list = chainCodeNormalization(chain_list) 

  normalized_data_chain[i] = normalized_list

  
print('Shortest list =', shortest_list)
print('Shortest list index =', shortest_list_index, 'Target:', data[shortest_list_index][cols-1])
print(len(chain_list))
#print(normalized_list)
print('\n')
print(normalized_data_chain, normalized_data_chain.shape)

(3352, 21)
Shortest list = 21
Shortest list index = 0 Target: 2.0
76


[[1. 1. 2. ... 3. 3. 1.]
 [1. 2. 2. ... 0. 0. 1.]
 [1. 1. 2. ... 0. 0. 0.]
 ...
 [1. 1. 1. ... 3. 3. 0.]
 [1. 1. 2. ... 3. 3. 0.]
 [1. 1. 2. ... 0. 0. 1.]] (3352, 21)


## Classifiers

In [ ]:
# Targets vector

targets = np.zeros((rows,))
for k in range(rows):
  targets[k] = data[k][cols-1]

print(targets)

[2. 6. 5. ... 7. 2. 1.]


In [ ]:
# Classifiers

#X = normalized_data_chain
X = hu_moments_matrix
#X = glcm_matrix
#X = binary_chain_data
y = targets

# SVM

#clf = svm.SVC(gamma = 'auto')

# SGDC Classifier

#clf = make_pipeline(StandardScaler(), SGDClassifier(max_iter=15, tol=1e-2))

# kNN Classifier

#clf = KNeighborsClassifier(n_neighbors=5)

# MLP Classifier

clf = MLPClassifier(random_state=1, max_iter=1000)

In [ ]:
# Setting the cross validation

sss = StratifiedShuffleSplit(n_splits=10 , test_size=0.2)

for train_index , test_index in sss.split(X,y):
  X_train , X_test = X[train_index] , X[test_index]
  y_train , y_test = y[train_index] , y[test_index]


In [ ]:
# Acc, std deviation and confusion matrix

std = []

mean_acc = 0

c_matrix = 0

for k, (train, test) in enumerate(sss.split(X, y)):
    clf.fit(X[train], y[train])

    predicted = clf.predict(X_test)

    acc = metrics.accuracy_score(y_test, predicted)

    std.append(acc)

    mean_acc = mean_acc + acc

    c_matrix = c_matrix + confusion_matrix(y_test, predicted)

    #print(confusion_matrix(y_test, predicted))
    #print(confusion_matrix(y_test, predicted).sum())

    #print("Accuracy: " f"{metrics.accuracy_score(y_test, predicted)}\n")

mean_c_matrix = c_matrix/10
for i in range(mean_c_matrix.shape[0]):
  for j in range(mean_c_matrix.shape[1]):
    c_matrix[i][j] = round(c_matrix[i][j])    

print('mean confusion matrix:') 
print(c_matrix/10) 
print('\n')
print('mean accuracy: ' , mean_acc/10) 

print('test samples: ' , c_matrix.sum()/10)

print('std deviation: ' , np.asarray(std).std())
print('\n\n')


# To plot the confusion matrix

plt.figure(figsize=(11,7))
sn.set(font_scale=1.8) 
sn.heatmap(c_matrix/10, annot=True, annot_kws={"size": 18}) # font size
plt.suptitle("Matriz de Confusão")
plt.xlabel("Predito")
plt.ylabel("Real")
plt.gcf().set_dpi(50)
plt.show()

In [ ]:
# Metrics

print(
    f"Classification report for classifier {clf}:\n"
    f"{metrics.classification_report(y_test, predicted)}\n"
)